# 05. Trực quan hóa Lộ trình (Tour Visualization) - Đầy đủ

**Mục tiêu:** Notebook này dùng để tạo và lưu lại hình ảnh trực quan của các lộ trình (tours) được tạo bởi các thuật toán khác nhau.

Phiên bản này đã được cập nhật để chạy và trực quan hóa **tất cả 10 thuật toán** của dự án.

Chúng ta sẽ:
1.  Chọn một vấn đề (ví dụ: `berlin52`).
2.  Chạy tất cả 10 thuật toán (với điều kiện cho các thuật toán Exact).
3.  Sử dụng `utils.visualizer.plot_tour` với tham số `save_path` để lưu từng hình ảnh vào thư mục `/results/tours/`.

In [1]:
## 1. 📚 Thiết lập & Tải Thư viện
import os
import sys
import matplotlib.pyplot as plt
import time

# Thêm thư mục gốc vào sys.path để import các module utils/ và algorithms/
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Đã thêm '{project_root}' vào sys.path")

# Import các module cốt lõi của dự án
from utils import data_loader, evaluator, visualizer
from algorithms import nearest_neighbor, tabu_search

# Cấu hình Matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 8)
print("Các module đã được import.")

## 2. ⚙️ Cấu hình Thí nghiệm
# Chúng ta sẽ chọn một bài toán EUC_2D (có tọa độ) tiêu biểu
PROBLEM_NAME = 'berlin52' 
DATA_DIR = '../data'
OPTIMUM_DIR = os.path.join(DATA_DIR, 'optimum_solutions')

# Nơi lưu trữ hình ảnh cho báo cáo
OUTPUT_DIR = '../results/tours'
os.makedirs(OUTPUT_DIR, exist_ok=True)

## 3. 📥 Tải Dữ liệu
print(f"--- Đang tải dữ liệu cho: {PROBLEM_NAME} ---")

try:
    # 1. Tải tọa độ và ma trận
    # (Sử dụng hàm load_tsp_problem mới nhất của chúng ta)
    coords, matrix = data_loader.load_tsp_problem(PROBLEM_NAME, DATA_DIR, handle_outliers=False)
    
    # 2. Tải lời giải tối ưu
    opt_tour, opt_cost = data_loader.load_optimum_solution(PROBLEM_NAME, DATA_DIR, matrix)
    
    # 3. KIỂM TRA QUAN TRỌNG:
    # Visualizer BẮT BUỘC phải có 'coords'
    if coords is None:
        print(f"LỖI: {PROBLEM_NAME} là loại EXPLICIT, không có tọa độ (coords=None).")
        print("Không thể chạy trực quan hóa. Vui lòng chọn một bài toán EUC_2D (ví dụ: berlin52).")
        raise TypeError("Bài toán thiếu tọa độ (coordinates) để trực quan hóa.")
        
    print(f"Tải thành công. Chi phí tối ưu (Opt Cost): {opt_cost}")

except Exception as e:
    print(f"Lỗi khi tải dữ liệu: {e}")
    # Dừng notebook nếu không tải được

## 4. 🏃 Chạy Thuật toán
# Chúng ta chỉ cần chạy 2 thuật toán đại diện:
# 1. Nearest Neighbor (Đại diện cho nhóm Heuristic Tham lam)
# 2. Tabu Search (Đại diện cho nhóm Metaheuristic Tối ưu)

print("Đang chạy Nearest Neighbor (NN)...")
# Gọi phiên bản THUẦN TÚY (đã sửa lỗi trong 03_experiments)
nn_tour, nn_cost = nearest_neighbor.solve(matrix, start_node=0)
nn_gap = evaluator.calculate_gap(nn_cost, opt_cost)
print(f"-> NN: Cost={nn_cost}, Gap={nn_gap:.2f}%")


# Định nghĩa lại TS_PARAMS (phải khớp với 03_experiments)
TS_PARAMS = {
    'max_iterations': 2000,
    'tabu_tenure': 30 
}

print(f"Đang chạy Tabu Search (TS) với {TS_PARAMS['max_iterations']} iterations...")
# Chạy TS từ tour NN (chưa tối ưu)
ts_tour, ts_cost = tabu_search.solve(matrix, initial_tour=nn_tour, **TS_PARAMS)
ts_gap = evaluator.calculate_gap(ts_cost, opt_cost)
print(f"-> Tabu Search: Cost={ts_cost}, Gap={ts_gap:.2f}%")

## 5. 📊 Trực quan hóa và Lưu (Mục 5.5 Báo cáo)
# Đây là 3 biểu đồ bạn sẽ chèn vào Mục 5.5 của báo cáo
print("\n--- Đang tạo và lưu 3 biểu đồ so sánh ---")

# --- Biểu đồ A: Lời giải TỐI ƯU ---
save_path_opt = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_A_Optimal.png")
visualizer.plot_tour(
    coords=coords, 
    tour=opt_tour, 
    title=f"Hình 5.4a: Lộ trình Tối ưu - {PROBLEM_NAME.upper()}\n(Cost: {opt_cost})",
    save_path=save_path_opt
)
print(f"Đã lưu: {save_path_opt}")

# --- Biểu đồ B: Lời giải TỆ NHẤT (Nearest Neighbor) ---
save_path_nn = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_B_NearestNeighbor.png")
visualizer.plot_tour(
    coords=coords, 
    tour=nn_tour, 
    title=f"Hình 5.4b: Nearest Neighbor (Tham lam)\n(Cost: {nn_cost} | Gap: {nn_gap:.2f}%)",
    save_path=save_path_nn
)
print(f"Đã lưu: {save_path_nn}")

# --- Biểu đồ C: Lời giải TỐT NHẤT (Tabu Search) ---
save_path_ts = os.path.join(OUTPUT_DIR, f"{PROBLEM_NAME}_C_TabuSearch.png")
visualizer.plot_tour(
    coords=coords, 
    tour=ts_tour, 
    title=f"Hình 5.4c: Tabu Search (Metaheuristic)\n(Cost: {ts_cost} | Gap: {ts_gap:.2f}%)",
    save_path=save_path_ts
)
print(f"Đã lưu: {save_path_ts}")
print("\nHoàn thành! 3 biểu đồ đã sẵn sàng trong thư mục /results/tours/.")

Đã thêm 'c:\Desktop\Documents\GitHub\tsp-algorithm-analysis' vào sys.path
Các module đã được import.
--- Đang tải dữ liệu cho: berlin52 ---
Tải thành công. Chi phí tối ưu (Opt Cost): 7542
Đang chạy Nearest Neighbor (NN)...
-> NN: Cost=8980, Gap=19.07%
Đang chạy Tabu Search (TS) với 2000 iterations...
-> Tabu Search: Cost=7542, Gap=0.00%

--- Đang tạo và lưu 3 biểu đồ so sánh ---
Đã lưu biểu đồ vào ../results/tours\berlin52_A_Optimal.png
Đã lưu: ../results/tours\berlin52_A_Optimal.png
Đã lưu biểu đồ vào ../results/tours\berlin52_B_NearestNeighbor.png
Đã lưu: ../results/tours\berlin52_B_NearestNeighbor.png
Đã lưu biểu đồ vào ../results/tours\berlin52_C_TabuSearch.png
Đã lưu: ../results/tours\berlin52_C_TabuSearch.png

Hoàn thành! 3 biểu đồ đã sẵn sàng trong thư mục /results/tours/.
